In [1]:
import pandas as pd
import numpy as np
import sys
import multiprocessing
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import matplotlib.ticker as mtick
import datetime as dt
from datetime import timedelta

import glob
import fathon
from fathon import fathonUtils as fu


import matplotlib as mpl
from multiprocessing import Pool, freeze_support, Manager

import itertools
colormap = plt.cm.RdBu
plt.style.use(os.path.join(mpl.get_configdir(),'latexstyle3.mplstyle'))

# import freqopttest.kernel as kernel

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [2]:
t71 = '/media/ak/T71/August11th2022Experiments/'
expInputFiles = os.path.join(t71,'ExperimentInputFiles' )
rx1Folder = os.path.join(expInputFiles, 'RX1')
os.listdir(expInputFiles)

['RX1',
 'FB1',
 'JB1',
 'G_1',
 'FV1',
 'TY1',
 'TU1',
 'DU1',
 'YM1',
 'XM1',
 'US1',
 'OE1',
 'KE1',
 'RXOld_do_I_delete']

In [3]:
symbols = ['RX1',
 'FB1',
 'JB1',
 'G_1',
 'FV1',
 'TY1',
 'TU1',
 'DU1',
 'YM1',
 'XM1',
 'US1',
 'OE1',
 'KE1']

In [4]:
print(symbols)

['RX1', 'FB1', 'JB1', 'G_1', 'FV1', 'TY1', 'TU1', 'DU1', 'YM1', 'XM1', 'US1', 'OE1', 'KE1']


In [5]:
rx_files = os.listdir(rx1Folder) # all the rx files
rx_files_volume = [f for f in rx_files if str('volume') in f]
rx_files_dollar = [f for f in rx_files if str('dollar') in f]
rx_files_tick = [f for f in rx_files if str('tick') in f]


In [6]:
# load example file 
idx =1 
idxFileLoc = os.path.join(rx1Folder, rx_files_volume[idx])
print(idxFileLoc)
df = pd.read_pickle(idxFileLoc)

/media/ak/T71/August11th2022Experiments/ExperimentInputFiles/RX1/RX1_volume_20180417_exp_df.pkl


In [7]:
df.columns.values

array(['micro_price', 'price_imbalance', 'pct_change_micro_price',
       'weighted_activity_spread', 'GK_vol', 'arrival_rates',
       'returns_normalised', 'returns_mix_max', 'skew', 'kurt',
       'median_traded_volume'], dtype=object)

In [8]:
df1, df2 = [x for _, x in df.groupby(df['GK_vol'] < np.median(df.GK_vol))]

In [9]:
df1.shape

(144657, 11)

In [10]:
df2.shape

(144657, 11)

In [11]:
df.shape

(289314, 11)

In [13]:
os.listdir('/media/ak/T71/August11th2022Experiments/HiLoData/')

['G_1_Hi_Vol_Regime', 'G_1_Lo_Vol_Regime']

In [16]:
pd.read_pickle('/media/ak/T71/August11th2022Experiments/HiLoData/G_1_Hi_Vol_Regime/G_1_tick_hi_regime20180608.pkl')

,micro_price,price_imbalance,pct_change_micro_price,weighted_activity_spread,GK_vol,arrival_rates,returns_normalised,returns_mix_max,skew,kurt,median_traded_volume
0,123.428832,-22.548579,0.000000,0.014368,0.000306,69.0,0.000000,0.000000,0.000000,0.000000e+00,3.5
1,123.408958,-18.875743,-0.000141,0.010254,0.000300,100.0,-0.284001,0.505877,0.000000,0.000000e+00,1.0
2,123.402989,23.684391,0.000103,0.011605,0.000298,77.0,0.209815,0.510257,0.000000,0.000000e+00,3.0
3,123.419123,32.862695,0.000163,0.009378,0.000292,63.0,0.331697,0.511339,0.000000,0.000000e+00,3.0
4,123.437050,31.378126,0.000077,0.007672,0.000293,42.0,0.155946,0.509780,0.000000,0.000000e+00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
19565,121.823353,-33.991260,0.000096,0.012916,0.000358,0.0,0.195210,0.510128,0.000169,7.721780e-12,1.0
19566,121.830362,-35.062900,0.000102,0.012084,0.000351,0.0,0.208168,0.510243,0.000171,7.836961e-12,1.0
19567,121.849302,4.091369,0.000195,0.014085,0.000349,0.0,0.396369,0.511912,0.000196,1.186395e-11,6.0
19568,121.859003,-12.840199,-0.000054,0.010793,0.000329,0.0,-0.107636,0.507441,0.000197,1.193158e-11,6.0


In [19]:
pd.read_pickle('/media/ak/T71/August11th2022Experiments/HiLoData/G_1_lo_Vol_Regime/G_1_tick_lo_regime20180608.pkl')

,micro_price,price_imbalance,pct_change_micro_price,weighted_activity_spread,GK_vol,arrival_rates,returns_normalised,returns_mix_max,skew,kurt,median_traded_volume
10,123.411330,33.700585,0.000027,0.006841,0.000280,42.0,0.054853,0.508883,0.000000,0.000000e+00,1.0
11,123.417930,37.564330,0.000050,0.005183,0.000278,67.0,0.103125,0.509311,0.000000,0.000000e+00,4.0
12,123.435378,6.906891,0.000153,0.008537,0.000280,60.0,0.310793,0.511153,0.000000,0.000000e+00,4.0
13,123.447509,2.679061,0.000089,0.010221,0.000268,30.0,0.181044,0.510002,0.000000,0.000000e+00,1.0
14,123.461694,39.809235,0.000179,0.010368,0.000265,25.0,0.364309,0.511628,0.000162,6.773327e-12,2.0
...,...,...,...,...,...,...,...,...,...,...,...
19584,121.820174,26.744774,0.000168,0.007900,0.000000,0.0,0.340572,0.511417,0.000182,9.283721e-12,1.0
19585,121.841032,26.452989,0.000161,0.007373,0.000000,0.0,0.327841,0.511304,0.000180,9.098108e-12,1.0
19586,121.860831,31.177243,0.000169,0.007579,0.000000,0.0,0.343876,0.511447,0.000180,9.047634e-12,1.0
19587,121.865897,7.596447,-0.000107,0.015516,0.000000,0.0,-0.216052,0.506480,0.000167,7.433050e-12,1.5


# variables
'micro_price',
 'price_imbalance',
 'pct_change_micro_price',
 'weighted_activity_spread',
 'GK_vol',
 'arrival_rates',
 'returns_normalised',
 'returns_mix_max',
 'skew',
 'kurt',
 'median_traded_volume'

In [ ]:
a1 = df1['price_imbalance']
b1 = df1['median_traded_volume']
c1 = df1['arrival_rates']

a2 = df2['price_imbalance']
b2 = df2['median_traded_volume']
c2 = df2['arrival_rates']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fathon
from fathon import fathonUtils as fu

In [ ]:
a1 = fu.toAggregated(a1)
b1 = fu.toAggregated(b1)
c1 = fu.toAggregated(c1)

a2 = fu.toAggregated(a2)
b2 = fu.toAggregated(b2)
c2 = fu.toAggregated(c2)

In [ ]:
pydcca = fathon.DCCA(a1, b1)

winSizes = fu.linRangeByStep(20, 1000, step=50)
polOrd = 1

n, F = pydcca.computeFlucVec(winSizes, polOrd=polOrd)

H, H_intercept = pydcca.fitFlucVec()

plt.plot(np.log(n), np.log(F), 'ro')
plt.plot(np.log(n), H_intercept+H*np.log(n), 'k-', label='H = {:.2f}'.format(H))
plt.xlabel('ln(n)', fontsize=14)
plt.ylabel('ln(F(n))', fontsize=14)
plt.title('DCCA', fontsize=14)
plt.legend(loc=0, fontsize=14)
plt.show()

limits_list = np.array([[20,120], [220,870]], dtype=int)
list_H, list_H_intercept = pydcca.multiFitFlucVec(limits_list)

clrs = ['k', 'b', 'm', 'c', 'y']
stls = ['-', '--', '.-']
plt.plot(np.log(n), np.log(F), 'ro')
for i in range(len(list_H)):
    n_rng = np.arange(limits_list[i][0], limits_list[i][1]+1, 50)
#     label='H tick = {:.2f}'.format(H_tick), color='red', linestyle = ':')
    plt.plot(np.log(n_rng), list_H_intercept[i]+list_H[i]*np.log(n_rng),
             clrs[i%len(clrs)]+stls[(i//len(clrs))%len(stls)], label='H_volume = {:.2f}'.format(list_H[i]), color='red', linestyle = ':')
plt.xlabel('ln(n)', fontsize=14)
plt.ylabel('ln(F(n))', fontsize=14)
plt.title('DCCA', fontsize=14)
plt.legend(loc=0, fontsize=14)
plt.show()

winSizes = fu.linRangeByStep(20, 1000, step=50)
polOrd = 1

n, rho = pydcca.computeRho(winSizes, polOrd=polOrd)

plt.plot(n, rho, 'ro')
plt.ylim(-1, 1)
plt.xlabel('n', fontsize=14)
plt.ylabel('$\\rho_{DCCA}$', fontsize=14)
plt.title('rhoDCCA', fontsize=14)
plt.show()

pythresh = fathon.DCCA()

L = 300
winSizes = fu.linRangeByStep(4, 100, step=1)
nSim = 100
confLvl = 0.95
polOrd = 1

n, cInt1, cInt2 = pythresh.rhoThresholds(L, winSizes, nSim, confLvl, polOrd=polOrd, verbose=True)

plt.plot(n, cInt1, 'r-')
plt.plot(n, cInt2, 'b-')
plt.ylim(-1, 1)
plt.xlabel('n', fontsize=14)
plt.show()

In [ ]:
pydcca = fathon.DCCA(a2, b2)

winSizes = fu.linRangeByStep(20, 1000, step=50)
polOrd = 1

n, F = pydcca.computeFlucVec(winSizes, polOrd=polOrd)

H, H_intercept = pydcca.fitFlucVec()

plt.plot(np.log(n), np.log(F), 'ro')
plt.plot(np.log(n), H_intercept+H*np.log(n), 'k-', label='H = {:.2f}'.format(H))
plt.xlabel('ln(n)', fontsize=14)
plt.ylabel('ln(F(n))', fontsize=14)
plt.title('DCCA', fontsize=14)
plt.legend(loc=0, fontsize=14)
plt.show()

limits_list = np.array([[20,120], [220,870]], dtype=int)
list_H, list_H_intercept = pydcca.multiFitFlucVec(limits_list)

clrs = ['k', 'b', 'm', 'c', 'y']
stls = ['-', '--', '.-']
plt.plot(np.log(n), np.log(F), 'ro')
for i in range(len(list_H)):
    n_rng = np.arange(limits_list[i][0], limits_list[i][1]+1, 50)
#     label='H tick = {:.2f}'.format(H_tick), color='red', linestyle = ':')
    plt.plot(np.log(n_rng), list_H_intercept[i]+list_H[i]*np.log(n_rng),
             clrs[i%len(clrs)]+stls[(i//len(clrs))%len(stls)], label='H_volume = {:.2f}'.format(list_H[i]), color='red', linestyle = ':')
plt.xlabel('ln(n)', fontsize=14)
plt.ylabel('ln(F(n))', fontsize=14)
plt.title('DCCA', fontsize=14)
plt.legend(loc=0, fontsize=14)
plt.show()

winSizes = fu.linRangeByStep(20, 1000, step=50)
polOrd = 1

n, rho = pydcca.computeRho(winSizes, polOrd=polOrd)

plt.plot(n, rho, 'ro')
plt.ylim(-1, 1)
plt.xlabel('n', fontsize=14)
plt.ylabel('$\\rho_{DCCA}$', fontsize=14)
plt.title('rhoDCCA', fontsize=14)
plt.show()

pythresh = fathon.DCCA()

L = 300
winSizes = fu.linRangeByStep(4, 100, step=1)
nSim = 100
confLvl = 0.95
polOrd = 1

n, cInt1, cInt2 = pythresh.rhoThresholds(L, winSizes, nSim, confLvl, polOrd=polOrd, verbose=True)

plt.plot(n, cInt1, 'r-')
plt.plot(n, cInt2, 'b-')
plt.ylim(-1, 1)
plt.xlabel('n', fontsize=14)
plt.show()

In [22]:
## plotting 